<a href="https://colab.research.google.com/github/LCaravaggio/EscueladeInnovacion/blob/main/OCR_y_An%C3%A1lisis_en_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pytesseract

In [4]:
import pytesseract
import requests

In [10]:
link='https://www.senado.gob.ar/prensa/adjunto/descargarArchivo/1726'
raw=""
ty="PdfReader"
pdf = requests.get(link)
my_raw_data = pdf.content

In [14]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.0 MB/s eta 0:00:00


In [15]:
from io import BytesIO
import PyPDF2
if pdf.status_code==200:
       with BytesIO(my_raw_data) as data:
         read_pdf = PyPDF2.PdfReader(data)
         for page in range(len(read_pdf.pages)):
           raw+=read_pdf.pages[page].extract_text()

In [16]:
raw

'“2024 – Año de la Defensa de la V ida, la Libertad y la P ropiedad”  \n1º de marzo  de 202 4 Asamblea Legislativa  Pág. 1 \nDirección General de Taquígrafos  \n \n \nASAMBLEA \nLEGISLATIVA  \nPERÍODO 14 2º \n1° DE MARZO  DE 202 4 \nPRESIDE NCIA DE LA SEÑORA VICEPRESIDENTE  DE L A NACIÓN, \nDOCTORA  VICTORIA VILLARRUEL  \nSE ENCUENTRAN PRES ENTES  EN EL ESTRADO DE LA PRESIDENCIA EL \nSEÑOR PRESIDENTE DE LA HONORABLE CÁMARA DE DIPUTADOS  DE LA \nNAC IÓN, DIPUTAD O MARTÍN MENEM  Y EL SEÑOR PRESIDENTE \nPRO VISIONAL DEL HONORABLE SENADO  DE LA NACIÓN, SENADOR \nBARTOLOMÉ ESTEBAN ABDALA  \nSECRETARIOS:  \nSEÑOR AGUSTÍN GIUSTINIAN , SECRETARIO DEL HONORABLE SE NADO,  \nY SEÑOR TOMÁS FIGUEROA , SECRETARIO DE LA HONORABLE CÁMARA \nDE DIPUTADOS  Pendiente de aprobación por parte de la Honorable  Cámara  \n(artículo  32, inciso m del Reglamento  del Senado de la Nación ) \n “2024 – Año de la Defensa de la V ida, la Libertad y la P ropiedad”  \n1º de marzo  de 202 4 Asamblea Legislativa  Pág. 2 